In [7]:
import asyncio
import websockets
import json
import pandas as pd
from csv import writer
from tqdm import tqdm
from datetime import datetime, timedelta
import numpy as np

In [8]:
msg = \
{
  "jsonrpc" : "2.0",
  "id" : 8772,
  "method" : "public/get_order_book",
  "params" : {
    "instrument_name" : "BTC-PERPETUAL",
    "depth" : 10,
  }
}

async def call_api(msg):
    async with websockets.connect('wss://test.deribit.com/ws/api/v2') as websocket:
        await websocket.send(msg)
        while websocket.open:
            response = await websocket.recv()
            return response


In [ ]:
keys = ['timestamp', 'best_bid_price', 'best_ask_price', 'estimated_delivery_price', 'index_price', 'mark_price', 'open_interest', 'settlement_price', 'change_id']
columns = keys + [x for i in range(10) for x in [f"ap_{i}", f"av_{i}", f"bp_{i}", f"bv_{i}"]]

old_lob_state = None

with open('data2.csv', 'w') as f:
    
    w = writer(f)
    w.writerow(columns)
    
    for i in tqdm(range(100000)):

        while True:
            data = await call_api(json.dumps(msg))
            data = json.loads(data)
            new_lob_state = [y for x in data['result']['bids'] + data['result']['asks'] for y in x]
            if old_lob_state != new_lob_state: break
                
        old_lob_state = new_lob_state
        
        vals = [data['result'][k] for k in keys]

        best_bid_price = data['result']['best_bid_price']
        best_ask_price = data['result']['best_ask_price']
        
        aps = [a[0] for a in data['result']['asks']]
        avs = [a[1] for a in data['result']['asks']]
        bps = [b[0] for b in data['result']['bids']]
        bvs = [b[1] for b in data['result']['bids']]

        vals += [x for ap, av, bp, bv in zip(aps, avs, bps, bvs) for x in [ap, av, bp, bv]]
        w.writerow(vals)

    f.close()

  0%|          | 453/100000 [19:05<82:27:15,  2.98s/it] 